In [26]:
# Libraries
import os
import json
import folium
import requests
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Show current working directory
print(os.getcwd())

/Users/ivesbrunner/Documents/Studium/01_Bachelor/04_Semester/04_ScientificProgramming/scientific_programming/Week_04/challenge


In [27]:
# Overpass API URL
url = 'http://overpass-api.de/api/interpreter'

# Overpass turbo query
query = f"""
        [out:json];
        area["ISO3166-1"="CH"][admin_level=2];
        node ["amenity"="fuel"](area);
        out;"""

# Web API request
r = requests.get(url, params={'data': query})
data = r.json()['elements']

with open('petrol_stations.json', 'w') as json_file:
    json.dump(data, json_file)

df = pd.json_normalize(data)
df.head()

df.to_csv('petrol_stations.csv', index=False)


In [28]:
tab = pd.pivot_table(df[['id', 'tags.name']],
                     index=['id'],
                     values=['tags.name'],
                     aggfunc=('count'))

# sort by count
tabSorted = tab.sort_values(by=('id'), ascending=True)
tabSorted


,tags.name
id,
734402,1
1228781,0
5702030,1
5702071,1
8082505,0
...,...
11577930778,1
11660181226,1
11662707353,1


In [29]:
# Subset of petrol stations by brand
locations = df[["lat", 
                "lon", 
                "tags.brand", 
                "tags.shop"]].loc[df["tags.brand"].isin(['Migrol', 
                                                         'Agip', 
                                                         'Shell'])]
print(locations.head(5))

# Create map
map = folium.Map(location=[locations.lat.mean(), 
                           locations.lon.mean()], 
                 zoom_start=8, 
                 control_scale=True)

# Add maker symbols
for index, location_info in locations.iterrows():
    folium.Marker([location_info["lat"], 
                   location_info["lon"]], 
                   popup=location_info["tags.brand"]).add_to(map)

# Plot map
map

          lat       lon tags.brand tags.shop
8   47.686119  8.621418      Shell       NaN
19  47.495696  8.712981      Shell       NaN
23  47.381593  8.509545      Shell       NaN
28  47.381110  8.492148     Migrol       NaN
32  47.337432  8.530242      Shell       NaN
